<a href="https://colab.research.google.com/github/pacozaa/LLM-Paper-To-Code/blob/main/DynamicCheatSheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
#Install Dependency
!pip install azure-ai-inference datasets

In [95]:
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage
from azure.core.credentials import AzureKeyCredential
from google.colab import userdata
from datasets import load_dataset

import re
import json

In [96]:
model_name = "openai/gpt-4.1"#openai/gpt-4.1 , phi-4
temperature=1.0
top_p=1.0
github_token=userdata.get('GITHUB_TOKEN') #GitHub Token

In [97]:
endpoint = "https://models.github.ai/inference"

token = github_token

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

In [98]:
# @title

generator_template="""
# GENERATOR (PROBLEM SOLVER)

Instruction: You are an expert problem-solving assistant tasked with analyzing and solving various questions using a combination of your expertise and provided reference materials. Each task will include:
1. A specific question or problem to solve
2. A cheatsheet containing relevant strategies, patterns, and examples from similar problems

---

## 1. ANALYSIS & STRATEGY

- Carefully analyze both the question and cheatsheet before starting
- Search for and identify any applicable patterns, strategies, or examples within the cheatsheet
- Create a structured approach to solving the problem at hand
- Review and document any limitations in the provided reference materials

## 2. SOLUTION DEVELOPMENT

- Present your solution using clear, logical steps that others can follow and review
- Explain your reasoning and methodology before presenting final conclusions
- Provide detailed explanations for each step of the process
- Check and verify all assumptions and intermediate calculations

## 3. FINAL ANSWER FORMAT

Start answering with `## Reasoning steps:` and end with

## Final answer:
```json

````

N.B. Make sure that the final answer is properly wrapped inside the ```json ``` block.


Example:
Puzzle: There are 2 houses, numbered 1 to 2 from left to right.
Each house is occupied by a different person.
Each house has a unique attribute for each of the following characteristics:
- Each person has a unique name: **Arnold, Eric**
- People own unique car models: **ford f150, tesla model 3**
- The people keep unique animals: **cat, horse**

**Clues**:
1. Eric is directly left of the person who owns a Tesla Model 3.
2. The person who keeps horses is in the first house.

Answer:
## Reasoning steps:

From Clue 1, we know that Eric is to the left of someone, so he must be the owner of House 1 because House 2 is the rightmost house.
Additionally, we know that the person in House 2 must be Arnold, and he owns a Tesla Model 3. Thus, Eric owns a Ford F150.
From Clue 2, we know that Eric keeps horses in House 1, which means the other house must keep cats. Finally, we arrive at the unique solution to this puzzle.
The solution is presented in table format:

## Final answer:

```json
{
   "header":[
      "Houses",
      "Name",
      "CarModel",
      "Animal"
   ],
   "rows":[
      [
         "1",
         "Eric",
         "ford f150",
         "horse"
      ],
      [
         "2",
         "Arnold",
         "tesla model 3",
         "cat"
      ]
   ]
}
```

-----

CHEATSHEET:
'''
[[CHEATSHEET]]
'''

-----
-----

Now it is time to solve the following question.

CURRENT INPUT:
'''
[[QUESTION]]
'''
"""
cheatsheet_template="""
# CHEATSHEET REFRENCE CURATOR

#### 1. Purpose and Goals
As the Cheatsheet Curator, you are tasked with creating a continuously evolving reference designed to help solve a wide variety of tasks, including algorithmic challenges, debugging, creative writing, and more. The cheatsheet's purpose is to consolidate verified solutions, reusable strategies, and critical insights into a single, well-structured resource.

- The cheatsheet should include quick, accurate, reliable, and practical solutions to a range of technical and creative challenges.
- After seeing each input, you should improve the content of the cheatsheet, synthesizing lessons, insights, tricks, and errors learned from past problems and adapting to new challenges.

---

#### 2. Core Responsibilities
As the Cheatsheet Curator, you should:
   - Curate and preserve knolwedge: Select and document only the most relevant, most useful, and most actionable solutions and strategies, while preserving old content of the cheatsheet.
   - Maintain accuracy: Ensure that all entries in the cheatsheet are accurate, clear, and well-contextualized.
   - Refine and update content: Continuously update and improve the content of the cheatsheet by incorporating new insights and solutions, removing repetitions or trivial information, and adding efficient solutions.
   - Ensure practicality and comprehensiveness: Provide critical and informative examples, as well as efficient code snippets and actionable guidelines.

Before updating the cheatsheet, however, you should first assess the correctness of the provided solution and strategically incorporate code blocks, insights, and solutions into the new cheatsheet. Always aim to preserve and keep correct, useful, and illustrative solutions and strategies for future cheatsheets.

---

#### 3. Principles and Best Practices
1. Accuracy and Relevance:
   - Only include solutions and strategies that have been tested and proven effective.
   - Clearly state any assumptions, limitations, or dependencies (e.g., specific Python libraries or solution hacks).
   - For computational problems, encourage Python usage for more accurate calculations.

2. Iterative Refinement:
   - Continuously improve the cheatsheet by synthesizing both old and new solutions, refining explanations, and removing redundancies.
   - Rather than deleting old content and writing new content each time, consider ways to maintain table content and synthesize information from multiple solutions.
   - After solving a new problem, document any reusable codes, algorithms, strategies, edge cases, or optimization techniques.

3. Clarity and Usability:
   - Write concise, actioanble, well-structured entries.
   - Focus on key insights or strategies that make solutions correct and effective.

4. Reusability:
   - Provide clear solutions, pseudocodes, and meta strategies that are easily adaptable to different contexts.
   - Avoid trivial content; focus on non-obvious, critical solution details and approaches.
   - Make sure to add as many examples as you can in the cheatsheet.
   - Any useful, efficient, generalizable, and illustrative solutions to the previous problems should be included in the cheatsheet.

---

#### 4. Cheatsheet Structure
The cheatsheet can be divided into the following sections:

1. Solutions, Implementation Patterns, and Code Snippets:
   - Document reusable code snippets, algorithms, and solution templates.
   - Include descriptions, annotated examples, and potential pitfalls, albeit succinctly.

2. [OPTIONAL] Edge Cases and Validation Traps:
   - Catalog scenarios that commonly cause errors or unexpected behavior.
   - Provide checks, validations, or alternative approaches to handle them.

3. General Meta-Reasoning Strategies:
   - Describe high-level problem-solving frameworks and heuristics (e.g., use Python to solve heuristic problems; in bipartite graphs, max matching = min vertex cover, etc.)
   - Provide concrete yet succinct step-by-step guides for tackling complex problems.

4. Implement a Usage Counter
   - Each entry must include a usage count: Increase the count every time a strategy is successfully used in problem-solving.
   - Use the count to prioritize frequently used solutions over rarely applied ones.

---

#### 5. Formatting Guidelines
Use the following structure for each memory item:

```
## Memory Item 1
** Count:  [Number of times this strategy has been used to solve a problem.]

### Description
[Briefly describe the problem context, purpose, and key aspects of the solution.] (Refence: Q1, Q2, Q6, etc.)

### Example
[Provide a well-documented code snippet, worked-out solution, or efficient strategy.]

---



## Memory Item 2
[...]

---

[...]

## Memory Item N
[...]

---

```

- Tagging: Use references like `(Q14)` or `(Q22)` to link entries to their originating contexts.
- Grouping: Organize entries into logical sections and subsections.
- Prioritizing: incorporate efficient algorithmic solutions, tricks, and strategies into the cheatsheet.
- Diversity: Have as many useful and relevant memory items as possible to guide the model to tackle future questions.

N.B. Keep in mind that once the cheatsheet is updated, any previous content not directly included will be lost and cannot be retrieved. Therefore, make sure to explicitly copy any (or all) relevant information from the previous cheatsheet to the new cheatsheet!!!

---

#### 6. Cheatsheet Template
Use the following format for creating and updating the cheatsheet:

NEW CHEATSHEET:
```
<cheatsheet>

## Version: [Version Number]

SOLUTIONS, IMPLEMENTATION PATTERNS, AND CODE SNIPPETS
## Memory Item 1
[...]

---

## Memory Item 2
[...]

---

GENERAL META-REASONING STRATEGIES
## Memory Item N
[...]

---

</cheatsheet>
```

N.B. Make sure that all information related to the cheatsheet is wrapped inside the <cheatsheet> block. The cheatsheet can be as long as circa 2000-2500 words.

-----
-----

## PREVIOUS CHEATSHEET

[[PREVIOUS_CHEATSHEET]]

-----
-----

## CURRENT INPUT

[[QUESTION]]

-----
-----

## MODEL ANSWER TO THE CURRENT INPUT

[[MODEL_ANSWER]]
"""



In [99]:
# @title
def get_generator_prompt(input_txt, generator_cheatsheet_content):
    return generator_template.replace("[[QUESTION]]", input_txt).replace("[[CHEATSHEET]]", generator_cheatsheet_content)

def get_curator_prompt(input_txt, generator_output, current_cheatsheet):
    return cheatsheet_template.replace("[[QUESTION]]", input_txt).replace("[[MODEL_ANSWER]]", generator_output).replace("[[PREVIOUS_CHEATSHEET]]", current_cheatsheet)

def llm_generator(input, cheatsheet):
    generate_messages = [
        UserMessage(get_generator_prompt(input, cheatsheet))
    ]
    response = client.complete(
        messages=generate_messages,
        temperature=temperature,
        top_p=top_p,
        model=model_name,
    )
    answer = response.choices[0].message.content
    return answer

def llm_curator(input, output, cheatsheet):
    curator_messages = [
        UserMessage(get_curator_prompt(input, output, cheatsheet))
    ]
    response = client.complete(
        messages=curator_messages,
        temperature=temperature,
        top_p=top_p,
        model=model_name,
    )
    answer = response.choices[0].message.content
    return answer

In [100]:
"""
This file contains the functions to extract the final answer, cheatsheet and solution evaluation from model responses.

The functions are:
* extract_answer(response: str) -> str: Extracts the final answer from the model response.
* extract_cheatsheet(response: str, old_cheatsheet: str) -> str: Extracts the cheatsheet from the model response.
* extract_solution(response: str, header: str = "SOLUTION EVALUATION:", error_message : str = "No solution evaluation found") -> str: Extracts the solution evaluation from the model response.

Additional functions can be added as needed.
"""

def extract_answer(
    response: str,
) -> str:
    """
    Extracts the final answer from the model response.

    Arguments:
        response : str : The response from the model.

    Returns:
        str : The extracted final answer (if not found, returns "No final answer found").
    """
    if "<answer>" in response:
        # <answer> (content) </answer>
        try:
            txt = response.split("<answer>")[-1].strip()
            txt = txt.split("</answer>")[0].strip()
            return txt
        except:
            return "No final answer found"
    else:
        if not("FINAL ANSWER" in response):
            return "No final answer found"
        try:
            response = response.split("FINAL ANSWER")[-1].strip()
            if response[0] == ":":
                response = response[1:].strip()

            # First decide whether to split by "```" or "'''" based on the presence of "```" or "'''"
            idx_1 = response.find("'''")
            idx_2 = response.find("```")
            if min(idx_1, idx_2) != -1:
                if idx_1 < idx_2:
                    response = response.split("'''")[1].strip()
                else:
                    response = response.split("```")[1].strip()
            else:
                if idx_1 == -1:
                    response = response.split("```")[1].strip()
                else:
                    response = response.split("'''")[1].strip()

            # Special case for P3-Test task: If the first line contains "python" then remove it
            if response.split("\n")[0].strip().lower() == "python":
                response = "\n".join(response.split("\n")[1:]).strip()
            return response
        except:
            return "No final answer found"


def extract_cheatsheet(
    response: str,
    old_cheatsheet: str,
) -> str:
    """
    Extracts the cheatsheet from the model response.

    Arguments:
        response : str : The response from the model.
        old_cheatsheet : str : The old cheatsheet to return if the new one is not found.

    Returns:
        str : The extracted cheatsheet (if not found, returns the old cheatsheet).
    """
    response = response.strip()
    # <cheatsheet> (content) </cheatsheet>
    if "<cheatsheet>" in response:
        try:
            txt = response.split("<cheatsheet>")[1].strip()
            txt = txt.split("</cheatsheet>")[0].strip()
            return txt
        except:
            return old_cheatsheet
    else:
        return old_cheatsheet


def extract_solution(
    response: str,
    header: str = "SOLUTION EVALUATION:",
    error_message : str = "No solution evaluation found",
) -> str:
    """
    Extracts the solution evaluation from the model response.

    Arguments:
        response : str : The response from the model.
        header : str : The header to search for the solution evaluation.
        error_message : str : The error message to return if the solution evaluation is not found.

    Returns:
        str : The extracted solution evaluation (if not found, returns the error message).
    """
    response = response.strip()
    try:
        txt = response.split(header)[1]
        try:
            txt = txt.split("'''")[1].strip()
        except:
            return txt.strip()
    except:
        return response
        # return error_message
    return txt


In [101]:
# @title

def extract_final_answer_regex(answer):
    match = re.search(r'## Final answer:\s*(.*)', answer, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return None

def extract_json_from_string(input_string):
    """
    Extract JSON from a string that contains ```json``` code blocks.

    Args:
        input_string (str): Input string that may contain JSON code blocks

    Returns:
        list: List of parsed JSON objects/dictionaries, or empty list if none found
    """
    # Regular expression pattern to match JSON code blocks
    pattern = r'```json\n(.*?)\n```'

    # Find all matches in the input string
    matches = re.findall(pattern, input_string, re.DOTALL)

    json_objects = []

    for match in matches:
        try:
            # Parse the JSON string into a Python object
            json_obj = json.loads(match.strip())
            json_objects.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
            print(f"Problematic JSON string: {match}")
            continue

    return json_objects


def find_name_column_index(answer):
    """Helper function to find the name column index in the header"""
    if 'header' in answer:
        try:
            return answer['header'].index('name')
        except (ValueError, IndexError):
            return None
    return None

def normalize_answer(answer, for_comparison=False):
    """
    Unified function to normalize answers for either counting or comparison.

    Args:
        answer: The answer to normalize
        for_comparison: If True, returns normalized rows for comparison.
                       If False, returns JSON string for counting duplicates.
    """
    if isinstance(answer, dict) and 'rows' in answer:
        name_idx = find_name_column_index(answer)

        if for_comparison:
            # For comparison: normalize all values and sort rows
            normalized_rows = []
            for row in answer['rows']:
                if name_idx is not None:
                    # Use the found name column index
                    normalized_row = []
                    for i, value in enumerate(row):
                        normalized_row.append(str(value).replace(" ", "").lower())
                    normalized_rows.append(tuple(normalized_row))
                else:
                    # Fallback: normalize all values
                    normalized_rows.append(tuple(str(val).replace(" ", "").lower() for val in row))

            # Sort rows by name for consistent comparison
            normalized_rows.sort()
            return normalized_rows
        else:
            # For counting: create dictionary keyed by name
            rows_by_name = {}
            for row in answer['rows']:
                if name_idx is not None:
                    name = row[name_idx]
                    rows_by_name[name] = tuple(row)  # Convert to tuple for hashability
                else:
                    # If no name column found, use second column (common pattern) or whole row
                    if len(row) > 1:
                        rows_by_name[row[1]] = tuple(row)
                    else:
                        rows_by_name[str(row)] = tuple(row)
            return json.dumps(rows_by_name, sort_keys=True)
    else:
        if for_comparison:
            return str(answer).replace(" ", "").lower()
        else:
            return str(answer)


In [102]:
# @title
# Intermediate Value
cheatsheet = ""
cheatsheet_list = [cheatsheet]

In [103]:
dataset = load_dataset("allenai/ZebraLogicBench-private", "grid_mode", split="test")
grid_size = "4*4"
ds_size = 5
filtered_dataset = dataset.filter(lambda x: x["size"] == grid_size).select(range(ds_size)).shuffle(seed=42)
display(filtered_dataset)
# https://huggingface.co/datasets/allenai/ZebraLogicBench-private
# Columns: id, size, puzzle, solution

Dataset({
    features: ['id', 'size', 'puzzle', 'solution', 'created_at'],
    num_rows: 5
})

In [104]:
# @title
filtered_dataset[0]['solution']

{'header': ['House', 'Name', 'Cigar', 'FavoriteSport', 'Drink'],
 'rows': [['1', 'Alice', 'prince', 'soccer', 'water'],
  ['2', 'Arnold', 'blue master', 'tennis', 'coffee'],
  ['3', 'Eric', 'dunhill', 'basketball', 'tea'],
  ['4', 'Peter', 'pall mall', 'swimming', 'milk']]}

In [105]:
is_correct_list = []


# Get total number of items for progress tracking
total_items = len(filtered_dataset)
print(f"Starting processing of {total_items} items...")

for index, item in enumerate(filtered_dataset):
    # Calculate progress percentage
    progress = (index + 1) / total_items * 100
    print(f"\n--- Processing item {index + 1}/{total_items} ({progress:.1f}%) ---")

    puzzle_id = item["id"]
    puzzle_input = item["puzzle"]

    solution = item["solution"]
    solution_normalized = normalize_answer(solution, for_comparison=True)

    generator_output = llm_generator(puzzle_input, cheatsheet)
    generator_output_extracted = extract_final_answer_regex(generator_output)

    if generator_output_extracted is None:
        print(f"❌ Warning: Could not extract answer from sample {puzzle_id} (Index: {index})")
        is_correct_list.append({
            "id": puzzle_id,
            "is_correct": False,
            "is_extracted": False,
            "index": index,
            "generator_output":generator_output
        })
    else:
        # print(f"✅ Extracted answer: {generator_output_extracted}")
        clean_generator_output_extracted = generator_output_extracted.lower().replace(" ", "")
        json_object_generator_output_extracted = extract_json_from_string(clean_generator_output_extracted)

        answer_normalized = normalize_answer(json_object_generator_output_extracted[0], for_comparison=True)

        is_correct = answer_normalized == solution_normalized

        # print(f"solution:{solution}\nsolution_normalized:{solution_normalized}\n")
        # print(f"answer:{json_object_generator_output_extracted}\nanswer_normalized:{answer_normalized}")

        if is_correct:
            print(f"✅ SUCCESS: The {puzzle_id} answer is correct!")
            is_correct_list.append({
                "id": puzzle_id,
                "is_correct": True,
                "is_extracted": True,
                "index": index,
                "generator_output":generator_output,
                "answer_normalized":answer_normalized,
                "solution_normalized":solution_normalized
            })
        else:
            print(f"❌ The {puzzle_id} answer does NOT match the correct answer")
            is_correct_list.append({
                "id": puzzle_id,
                "is_correct": False,
                "is_extracted": True,
                "index": index,
                "generator_output":generator_output,
                "answer_normalized":answer_normalized,
                "solution_normalized":solution_normalized
            })

        cheatsheet_output = llm_curator(puzzle_input, generator_output, cheatsheet)
        new_cheatsheet = extract_cheatsheet(cheatsheet_output, cheatsheet)
        cheatsheet = new_cheatsheet
        cheatsheet_list.append(new_cheatsheet)
    print(f"✅ Processed item: {item['id']} (Index: {index})")

    # Optional: Add a break here for testing with a smaller subset
    # if index >= 4:  # Test with first 5 items
    #     break

# Calculate final statistics
correct_count = sum(1 for item in is_correct_list if item["is_correct"])
accuracy = correct_count / len(is_correct_list) * 100 if is_correct_list else 0

print(f"\n{'='*50}")
print(f"✅ Iteration complete! Processed {len(is_correct_list)} items")
print(f"📊 Results: {correct_count}/{len(is_correct_list)} correct ({accuracy:.1f}% accuracy)")
print(f"{'='*50}")

Starting processing of 5 items...

--- Processing item 1/5 (20.0%) ---
❌ The lgp-test-4x4-13 answer does NOT match the correct answer
✅ Processed item: lgp-test-4x4-13 (Index: 0)

--- Processing item 2/5 (40.0%) ---
❌ The lgp-test-4x4-21 answer does NOT match the correct answer
✅ Processed item: lgp-test-4x4-21 (Index: 1)

--- Processing item 3/5 (60.0%) ---
✅ SUCCESS: The lgp-test-4x4-16 answer is correct!
✅ Processed item: lgp-test-4x4-16 (Index: 2)

--- Processing item 4/5 (80.0%) ---
✅ SUCCESS: The lgp-test-4x4-3 answer is correct!
✅ Processed item: lgp-test-4x4-3 (Index: 3)

--- Processing item 5/5 (100.0%) ---
✅ SUCCESS: The lgp-test-4x4-27 answer is correct!
✅ Processed item: lgp-test-4x4-27 (Index: 4)

✅ Iteration complete! Processed 5 items
📊 Results: 3/5 correct (60.0% accuracy)


In [106]:
# is_correct_list[0]

{'id': 'lgp-test-4x4-13',
 'is_correct': False,
 'is_extracted': True,
 'index': 0,
 'generator_output': '## Reasoning steps:\n\nLet\'s carefully analyze the clues and map out the problem systematically, recording intermediate deductions.\n\n### 1. Entities\n- Houses: 1, 2, 3, 4\n- Names: Alice, Peter, Arnold, Eric\n- Cigars: prince, dunhill, blue master, pall mall\n- Sports: swimming, basketball, soccer, tennis\n- Drinks: coffee, water, milk, tea\n\n### 2. Clues Review & Initial Placements\n\n#### Assign known places directly from clues.\n\n- Clue 1: **Peter is in the fourth house.**  \n  ⇒ House 4: Peter\n\n- Clue 8: **The person who loves basketball is in the third house.**  \n  ⇒ House 3: Basketball\n\n- Clue 4: **The person who loves basketball is Eric.**  \n  ⇒ House 3: Eric\n\nSo:\n- House 3: Eric, Basketball\n- House 4: Peter\n\n#### Name + Cigar cross referencing\n\n- Clue 3: **Arnold is the person who smokes Blue Master.**  \n  ⇒ Arnold ↔ Blue Master\n\n- Clue 7: **The coffee

In [107]:
cheatsheet_list[2]

'## Version: 2.0\n\n---\n\nSOLUTIONS, IMPLEMENTATION PATTERNS, AND CODE SNIPPETS\n\n---\n\n## Memory Item 1\n**Count: 2**\n\n### Description\n**Logic puzzle (Zebra-type): Tabular setup and permutational search for multi-category assignment.**  \nEfficiently solve puzzles with multiple entities and mutually exclusive attributes using a table per house. Start with direct assignments from the clues, then use constraint propagation, permutation-based search, and elimination to fill remaining slots. (Reference: Q1, Q2)\n\n### Example\n\n**Step-by-step pattern:**\n1. Create a table with each house as a row; columns for each attribute.\n2. Insert all direct assignments ("X is Y", "X is in house N").\n3. Parse clues that relate categories ("A is B is C in the same house") and assign those together.\n4. For clues involving order ("A is left of B", "X is directly to the left of Y"), use indices.\n5. For clues like "N houses between", calculate allowed pairs (see Memory Item 2).\n6. Ensure unique